# Fine-tune Llama2 with LoRA for QA


reference code: https://deci.ai/blog/fine-tune-llama-2-with-lora-for-question-answering/

In [1]:
# import warnings
# warnings.filterwarnings('error', category=DeprecationWarning)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

Error [NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported] resolving:

1. pip install -U datasets
2. pip install fsspec==2023.9.2
3. restart the kernel of this jupyter notebook

https://stackoverflow.com/questions/77433096/notimplementederror-loading-a-dataset-cached-in-a-localfilesystem-is-not-suppor

In [3]:
# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
training_data = load_dataset(data_name, split="train")

# Model and tokenizer names
base_model_name = "/home/fanhuan/cache/llama-2-13b-chat-hf"
refined_model = "/home/fanhuan/cache/llama-2-13b-chat-hf-TF-LoRA"
cache_dir = "/data/fanhuan/cache/temp/13b-lora"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
#     device_map={"": 7}
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /home/fanhuan/cache/llama-2-13b-chat-hf and are newly initialized: ['model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.32.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.36.self_attn.rotary_emb.inv_freq', 'model.layers.33.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.35.self_attn.rotary_emb.inv_freq', 'model.layers.34.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.19.

How to use the neptune features in Transformers:

https://docs.neptune.ai/integrations/transformers/#__tabbed_2_1

In [5]:
# from transformers.integrations import NeptuneCallback
# import neptune

# run = neptune.init_run(
#     project="fhuang181/LoRA",
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZmI2ZTA2OC00ZGIxLTQ2NDktYTU4YS1jOWUyNWIwYmU3YWUifQ==",
# )

# neptune_callback = NeptuneCallback(run=run)

# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir=cache_dir,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    # very important setting for keep the disk space enough for further training
    save_total_limit = 1,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    # use the report_to parameter to avoid error in neptune stuff
    # report_to="none"
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,
#     callbacks=[neptune_callback]
)

fine_tuning.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.234000
50,1.550500
75,1.139100
100,1.350400
125,1.098200
150,1.276400
175,1.097900
200,1.375600
225,1.074900
250,1.431600


TrainOutput(global_step=250, training_loss=1.262870246887207, metrics={'train_runtime': 415.0121, 'train_samples_per_second': 2.41, 'train_steps_per_second': 0.602, 'total_flos': 1.679743262306304e+16, 'train_loss': 1.262870246887207, 'epoch': 1.0})

In [31]:
fine_tuning.model.save_pretrained(refined_model)
fine_tuning.tokenizer.save_pretrained(refined_model)

('/home/fanhuan/cache/llama-2-13b-chat-hf-TF-LoRA/tokenizer_config.json',
 '/home/fanhuan/cache/llama-2-13b-chat-hf-TF-LoRA/special_tokens_map.json',
 '/home/fanhuan/cache/llama-2-13b-chat-hf-TF-LoRA/tokenizer.model',
 '/home/fanhuan/cache/llama-2-13b-chat-hf-TF-LoRA/added_tokens.json',
 '/home/fanhuan/cache/llama-2-13b-chat-hf-TF-LoRA/tokenizer.json')

In [42]:
# Fine-tuned model
prompt = "How do I use the OpenAI API?"
pipe = pipeline(task="text-generation", model=fine_tuning.model, tokenizer=llama_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

<s>[INST] How do I use the OpenAI API? [/INST] The OpenAI API is a powerful tool that allows developers to access a wide range of AI models and capabilities. Here are some steps to help you get started with using the OpenAI API:

1. Sign up for an OpenAI account: To use the OpenAI API, you need to sign up for an OpenAI account. You can sign up for free on the OpenAI website.
2. Create a new project: Once you have an OpenAI account, you can create a new project. This will allow you to access the OpenAI API and start building your AI application.
3. Choose an AI model: The OpenAI API provides access to a wide range of AI models, including language models, computer vision models, and more. You can choose the model that best fits your needs.
4. Use the OpenAI API: Once you have chosen an


In [40]:
# Original model
prompt = "How do I use the OpenAI API?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I use the OpenAI API? [/INST] The OpenAI API is a powerful tool that allows developers to access a wide range of AI models and capabilities. Here are some steps to help you get started with the OpenAI API:

1. Sign up for an OpenAI account: To use the OpenAI API, you need to sign up for an OpenAI account. You can sign up for free on the OpenAI website.
2. Create a new project: Once you have an OpenAI account, you can create a new project. This will allow you to access the OpenAI API and start building your AI application.
3. Choose an AI model: The OpenAI API provides access to a wide range of AI models, including language models, computer vision models, and more. You can choose the model that best fits your needs.
4. Use the OpenAI API: Once you have chosen an A


### Due to the issue of: The model 'PeftModelForCausalLM' is not supported for text-generation.

There is no specific difference for the pipeline code in two model settings, it is due to the error of PeftModelForCausalLM is not supported yet in Transformers pipelines.

According to (https://huggingface.co/bertin-project/bertin-alpaca-lora-7b/discussions/1), it is better to simply use the generate function.

## Controlled generation, via generate function

In [19]:
# Generate Text - before fine-tune

cuda_name = 'cuda:0'
model = base_model
tokenizer = llama_tokenizer

text = "How do I use the OpenAI API?"
inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                           pad_token_id=tokenizer.eos_token_id,
                                           eos_token_id=tokenizer.eos_token_id,
                                           return_dict_in_generate=True, output_scores=True)

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
temp_output = tokenizer.decode(generated_tokens[0])

print(temp_output)

'[closed]\n\nThe OpenAI API appears to be unstable, but that doesn\'t seem to be the case. I\'ve been struggling with the OpenAI API for a few weeks and I can\'t figure out how to use it properly. I\'ve tried using the code from the OpenAI website but none of it works for me.\n\nThis is my first time using a machine learning API so I\'m open to suggestions.\n\nThe error I\'m seeing is "Not supported", so I\'m starting to assume it may be because I don\'t have the right permissions.\n\nAny advice would be helpful.\n\n[What I\'ve tried]\n\nI\'ve tried a few different things so far that I found on the OpenAI website but none of it works for me. I have a feeling it may be because I don\'t have the right permissions, but I\'m not sure.\n\nThe error I\'m seeing is "Not supported", and some tutorials I found online have this error as well. If you have experience working with OpenAI, please let me know what I\'m doing wrong.\n\nI\'ve tried using the Jupyter Notebook as well, and it does not wo

In [25]:
# Generate Text - after fine-tune

cuda_name = 'cuda:0'
model = fine_tuning.model
tokenizer = llama_tokenizer

text = "How do I use the OpenAI API?"
inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                           pad_token_id=tokenizer.eos_token_id,
                                           eos_token_id=tokenizer.eos_token_id,
                                           return_dict_in_generate=True, output_scores=True)

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
temp_output = tokenizer.decode(generated_tokens[0])

print(temp_output)


--------------------------

### OpenAI API endpoints

 - [GET] `https://api.openai.com/v1`
 - [POST] `https://api.openai.com/v1/{operation}`

### Authentication

 - API key: `Authorization: Bearer {API_KEY}`
 - Token: `Authorization: Bearer {TOKEN}`
 
### Operations

 - [GET] `https://api.openai.com/v1/models`
 - [GET] `https://api.openai.com/v1/models/{id}`
 - [POST] `https://api.openai.com/v1/models/{id}/parameters`
```
### Request body

 - [POST] `https://api.openai.com/v1/models/{id}/parameters`
 {
  "description": "My AI model",
  "model": {
    "model_name": "My model",
    "model_version": 1,
    "description": "My AI model",
    "class": "ai.Model",
    "type": "My model",
    "labels": [
      {
        "label": "My label",
        "description": "My label",
        "values": [
          [
            "My label value 1",
            "My label value 2"
          ]
        ]
      }
    ]
  }
}
```
This endpoint allows me to create, edit, delete models, including parameters. Fo

# Fine-grained performance analysis

In [44]:
import pandas as pd

df = pd.read_csv('dataset/implicit-hate-speech.csv')
df

,post,human_answer
0,and i will point it out here when u call whit...,Blacks blame whites
1,antiwhites expose their hatred of white kids ...,Anti-whites hate white people
2,interior sec y salazar pushing for hispanic n...,Minorities are taking over society
3,national socialism is race based which means ...,Minorities are subpar races
4,sez the cop hater in chief the terrorist said...,Blacks do the bidding of jews
...,...,...
95,why should the white race not enjoy a homeland...,White people are being replaced by minorities
96,wow robe sounds like a 10 year old stamping h...,Robert is insecure.
97,yea we were better offwith the floppy eared ga...,the group is wasting money.
98,you go guys white power,White people are powerful.


In [50]:
from tqdm import tqdm

cuda_name = 'cuda:0'

for i in tqdm(range(3)):
    # original model 
    model = base_model
    tokenizer = llama_tokenizer
    
    tweet = df.loc[0,'post']

    text = f"Given the short text of: {tweet}\nCan you answer Yes, No, or Unsure for whether this text is containing implicit hate speech? And then explain why in few setences."
    inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                               pad_token_id=tokenizer.eos_token_id,
                                               eos_token_id=tokenizer.eos_token_id,
                                               return_dict_in_generate=True, output_scores=True)

    input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
    generated_tokens = outputs.sequences[:, input_length:]
    temp_output = tokenizer.decode(generated_tokens[0])

    df.loc[i, 'llama2-13b-chat'] = temp_output
    
    # fine-tuned model, using LoRA
    model = fine_tuning.model
    tokenizer = llama_tokenizer
    
    tweet = df.loc[0,'post']

    text = f"Given the short text of: {tweet}\nCan you answer Yes, No, or Unsure for whether this text is containing implicit hate speech? And then explain why in few setences."
    inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                               pad_token_id=tokenizer.eos_token_id,
                                               eos_token_id=tokenizer.eos_token_id,
                                               return_dict_in_generate=True, output_scores=True)

    input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
    generated_tokens = outputs.sequences[:, input_length:]
    temp_output = tokenizer.decode(generated_tokens[0])

    df.loc[i, 'llama2-13b-chat-FT-LoRA'] = temp_output

100%|█████████████████████████████████████████████████████████████████████████████████| 3/3 [01:07<00:00, 22.49s/it]


In [54]:
df.to_csv('dataset/implicit-hate-speech.csv', index=False)